In [0]:
%pip install pypdf pandas sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 158.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 134.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 20.7 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
volume_dir = "dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/"
files = [f.path for f in dbutils.fs.ls(volume_dir) if f.path.endswith(".pdf")]
display(files)


_1
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_1.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_10.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_2.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_3.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_4.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_5.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_6.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_7.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_8.pdf
dbfs:/Volumes/catalog_anushka/pdf_fin_bronze/pdf_raw/monthly_bank_statement_9.pdf


In [0]:
import io
from pypdf import PdfReader

def extract_text_dbfs(dbfs_path):
    # Read binary data using Spark (Databricks-supported)
    binary_df = spark.read.format("binaryFile").load(dbfs_path)
    pdf_bytes = binary_df.collect()[0]["content"]

    # Convert to in-memory file
    pdf_stream = io.BytesIO(pdf_bytes)
    reader = PdfReader(pdf_stream)

    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"
    return text


In [0]:
docs = []

for f in files:
    text = extract_text_dbfs(f)
    docs.append({
        "filename": f.split("/")[-1],
        "text": text
    })

docs


[{'filename': 'monthly_bank_statement_1.pdf',
  'text': 'Monthly Bank Statement (Anonymized)\nAccount Type: Student Girokonto\nIBAN: DE76244482017137843165\nStatement Period: 01.01.2025 - 30.01.2025\nDate\nDescription\nCategory\nAmount\n01.01.2025\nStudent Salary\nEINNAHMEN\n+1,200.00 EUR\n02.01.2025\nMonthly Rent\nWOHNEN\n-350.00 EUR\n03.01.2025\nHealth Insurance\nVERSICHERUNG\n-140.00 EUR\n06.01.2025\nSupermarket\nLEBENSHALTUNG\n-50.00 EUR\n10.01.2025\nSupermarket\nLEBENSHALTUNG\n-45.00 EUR\n14.01.2025\nSupermarket\nLEBENSHALTUNG\n-55.00 EUR\n16.01.2025\nRestaurant & Cafés\nRESTAURANTS\n-138.00 EUR\n21.01.2025\nTravel (Flight/Train)\nREISEN\n-220.00 EUR\n14.01.2025\nPharmacy\nMOBILITÄT\n-39.74 EUR\n23.01.2025\nClothing\nSONSTIGES\n-9.48 EUR\n25.01.2025\nClothing\nMOBILITÄT\n-39.60 EUR\n19.01.2025\nMobile Plan\nMOBILITÄT\n-35.93 EUR\n\n'},
 {'filename': 'monthly_bank_statement_10.pdf',
  'text': 'Monthly Bank Statement (Anonymized)\nAccount Type: Student Girokonto\nIBAN: DE14523937786

In [0]:
df = spark.createDataFrame(docs)
display(df)


filename,text
monthly_bank_statement_1.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE76244482017137843165 Statement Period: 01.01.2025 - 30.01.2025 Date Description Category Amount 01.01.2025 Student Salary EINNAHMEN +1,200.00 EUR 02.01.2025 Monthly Rent WOHNEN -350.00 EUR 03.01.2025 Health Insurance VERSICHERUNG -140.00 EUR 06.01.2025 Supermarket LEBENSHALTUNG -50.00 EUR 10.01.2025 Supermarket LEBENSHALTUNG -45.00 EUR 14.01.2025 Supermarket LEBENSHALTUNG -55.00 EUR 16.01.2025 Restaurant & Cafés RESTAURANTS -138.00 EUR 21.01.2025 Travel (Flight/Train) REISEN -220.00 EUR 14.01.2025 Pharmacy MOBILITÄT -39.74 EUR 23.01.2025 Clothing SONSTIGES -9.48 EUR 25.01.2025 Clothing MOBILITÄT -39.60 EUR 19.01.2025 Mobile Plan MOBILITÄT -35.93 EUR"
monthly_bank_statement_10.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE14523937786229979949 Statement Period: 28.09.2025 - 27.10.2025 Date Description Category Amount 28.09.2025 Student Salary EINNAHMEN +1,200.00 EUR 29.09.2025 Monthly Rent WOHNEN -350.00 EUR 30.09.2025 Health Insurance VERSICHERUNG -140.00 EUR 03.10.2025 Supermarket LEBENSHALTUNG -50.00 EUR 07.10.2025 Supermarket LEBENSHALTUNG -45.00 EUR 11.10.2025 Supermarket LEBENSHALTUNG -55.00 EUR 13.10.2025 Restaurant & Cafés RESTAURANTS -64.00 EUR 18.10.2025 Travel (Flight/Train) REISEN -105.00 EUR 21.10.2025 Mobile Plan MOBILITÄT -22.43 EUR 12.10.2025 Clothing ONLINE -5.39 EUR 09.10.2025 Streaming MOBILITÄT -28.70 EUR"
monthly_bank_statement_2.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE32445058247297456566 Statement Period: 31.01.2025 - 01.03.2025 Date Description Category Amount 31.01.2025 Student Salary EINNAHMEN +1,200.00 EUR 01.02.2025 Monthly Rent WOHNEN -350.00 EUR 02.02.2025 Health Insurance VERSICHERUNG -140.00 EUR 05.02.2025 Supermarket LEBENSHALTUNG -50.00 EUR 09.02.2025 Supermarket LEBENSHALTUNG -45.00 EUR 13.02.2025 Supermarket LEBENSHALTUNG -55.00 EUR 15.02.2025 Restaurant & Cafés RESTAURANTS -74.00 EUR 20.02.2025 Travel (Flight/Train) REISEN -213.00 EUR 14.02.2025 Mobile Plan ONLINE -6.40 EUR 25.02.2025 Clothing ONLINE -13.55 EUR 24.02.2025 Mobile Plan MOBILITÄT -35.53 EUR 25.02.2025 Streaming MOBILITÄT -15.20 EUR"
monthly_bank_statement_3.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE29913544711953331616 Statement Period: 02.03.2025 - 31.03.2025 Date Description Category Amount 02.03.2025 Student Salary EINNAHMEN +1,200.00 EUR 03.03.2025 Monthly Rent WOHNEN -350.00 EUR 04.03.2025 Health Insurance VERSICHERUNG -140.00 EUR 07.03.2025 Supermarket LEBENSHALTUNG -50.00 EUR 11.03.2025 Supermarket LEBENSHALTUNG -45.00 EUR 15.03.2025 Supermarket LEBENSHALTUNG -55.00 EUR 17.03.2025 Restaurant & Cafés RESTAURANTS -99.00 EUR 22.03.2025 Travel (Flight/Train) REISEN -80.00 EUR 18.03.2025 Pharmacy ONLINE -21.08 EUR 27.03.2025 Clothing SONSTIGES -13.34 EUR 23.03.2025 Streaming MOBILITÄT -12.09 EUR"
monthly_bank_statement_4.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE44759000387301292676 Statement Period: 01.04.2025 - 30.04.2025 Date Description Category Amount 01.04.2025 Student Salary EINNAHMEN +1,200.00 EUR 02.04.2025 Monthly Rent WOHNEN -350.00 EUR 03.04.2025 Health Insurance VERSICHERUNG -140.00 EUR 06.04.2025 Supermarket LEBENSHALTUNG -50.00 EUR 10.04.2025 Supermarket LEBENSHALTUNG -45.00 EUR 14.04.2025 Supermarket LEBENSHALTUNG -55.00 EUR 16.04.2025 Restaurant & Cafés RESTAURANTS -129.00 EUR 17.04.2025 Mobile Plan MOBILITÄT -31.20 EUR 26.04.2025 Pharmacy SONSTIGES -22.31 EUR 19.04.2025 Taxi MOBILITÄT -23.40 EUR"
monthly_bank_statement_5.pdf,"Monthly Bank Statement (Anonymized) Account Type: Student Girokonto IBAN: DE77970537415979725633 Statement Period: 01.05.2025 - 30.05.2025 Date Description Category Amount 01.05.2025 Student Salary EINNAHMEN +1,200.00 EUR 02.05.2025 Monthly Rent WOHNEN -350.00 EUR 03.05.2025 Health Insurance VERSICHERUNG -140.00 EUR 06

In [0]:
df.write.format("delta").mode("overwrite").saveAsTable(
    "catalog_anushka.pdf_fin_bronze.pdf_raw"
)
